In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
movies=pd.read_csv('movies.csv')
ratings=pd.read_csv('ratings.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
#So what we see is movies dataframe consist of all movie info and what genre it have and rating consist of users rating to various movies.

In [6]:
#Now removing the unnecessary column like timestamp from ratings and genres from movies(although genres is a very important feature but it is for content based system where similarity is between movies and contents but right now we need similarity between users)

In [7]:
movies.drop('genres',axis=1,inplace=True)
ratings.drop('timestamp',axis=1,inplace=True)

In [8]:
#Now combining the both the dataframes on movieId

In [9]:
df=movies.merge(ratings,on='movieId')

In [10]:
df.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [11]:
#Now we can see for a particular movie many users had given different ratings

In [12]:
new_df=df.dropna(axis=0,subset=['title'])

In [13]:
#now creating a separate dataframe for total number of rating counts

In [14]:
count=(new_df.groupby('title')['rating'].count().reset_index())

In [15]:
count.head()

,title,rating
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [16]:
count['totalratingcount']=count['rating']
count.drop('rating',inplace=True,axis=1)

In [17]:
count.head()

,title,totalratingcount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [18]:
#Now we have rating count for each and every movie hence we can merge this dataframe to older datafrane

In [19]:
final=new_df.merge(count, left_on = 'title', right_on = 'title', how = 'left')
#Now this final dataframe consist of rating count and total rating count for every movie given by user

In [20]:
final.head()

,movieId,title,userId,rating,totalratingcount
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215


In [21]:
rating_threshold=50
final_1=final.query('totalratingcount >=@rating_threshold')
final_1.head()

,movieId,title,userId,rating,totalratingcount
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215


In [22]:
final_1.shape

(41362, 5)

In [23]:
features=final_1.pivot_table(values='rating', index='title', columns='userId').fillna(0)
features.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0


In [24]:
from scipy.sparse import csr_matrix
final_matrix=csr_matrix(features.values)

In [25]:
from sklearn.neighbors import NearestNeighbors

In [26]:
model_knn=NearestNeighbors(metric='cosine',algorithm='brute')

In [27]:
model_knn.fit(final_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [44]:
#Now finding the matching movies on the basis of index
query_index = np.random.choice(features.shape[0])

print(query_index)
distances, indices = model_knn.kneighbors(features.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

408


In [45]:
for i in range(0,len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(features.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, features.index[indices.flatten()[i]], distances.flatten()[i]))


Recommendations for Toy Story (1995):

1: Toy Story 2 (1999), with distance of 0.4273987396802844:
2: Jurassic Park (1993), with distance of 0.4343631959138433:
3: Independence Day (a.k.a. ID4) (1996), with distance of 0.43573830647233425:
4: Star Wars: Episode IV - A New Hope (1977), with distance of 0.4426118294200634:
5: Forrest Gump (1994), with distance of 0.4529040920598262:
